In [33]:
import numpy as np
import pandas as pd
import math

In [34]:
print(-(2/3*math.log(2/3,2)+1/3*math.log(1/3,2)))

0.9182958340544896


In [35]:
print(0.97-(2+3*0.91)/5)

0.02399999999999991


In [55]:
#Save dataset from pandas DataFrame
Data = {'District':['Suburban','Suburban','Rural','Urban','Urban','Urban','Rural','Suburban','Suburban','Urban','Suburban','Rural','Rural','Urban'],
        'House Type':['Detached','Detached','Detached','Semi-detached','Semi-detached','Semi-detached','Semi-detached','Terrace','Semi-detached','Terrace',
                      'Terrace','Terrace','Detached','Terrace'],
        'Income':['High','High','High','High','Low','Low','Low','High','Low','Low','Low','High','Low','High'],
        'Previous Customer':['No','Yes','No','No','No','Yes','Yes','No','No','No','Yes','Yes','No','Yes'],
        'Outcome':['Not responded','Not responded','Responded','Responded','Responded','Not responded','Responded','Not responded','Responded','Responded',
                   'Responded','Responded','Responded','Not responded']}

dataset = pd.DataFrame(Data)
print(dataset.info())
print()
print(dataset.head(14))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   District           14 non-null     object
 1   House Type         14 non-null     object
 2   Income             14 non-null     object
 3   Previous Customer  14 non-null     object
 4   Outcome            14 non-null     object
dtypes: object(5)
memory usage: 692.0+ bytes
None

    District     House Type Income Previous Customer        Outcome
0   Suburban       Detached   High                No  Not responded
1   Suburban       Detached   High               Yes  Not responded
2      Rural       Detached   High                No      Responded
3      Urban  Semi-detached   High                No      Responded
4      Urban  Semi-detached    Low                No      Responded
5      Urban  Semi-detached    Low               Yes  Not responded
6      Rural  Semi-detached    Low        

In [56]:
features = dataset[['District','House Type','Income','Previous Customer']]
target = dataset[['Outcome']]

print(features.head())

   District     House Type Income Previous Customer
0  Suburban       Detached   High                No
1  Suburban       Detached   High               Yes
2     Rural       Detached   High                No
3     Urban  Semi-detached   High                No
4     Urban  Semi-detached    Low                No


In [51]:
#Original data entropy -> MO
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True) #remove redundant data
    entropy = 0
    for i in range(len(elements)):
        entropy += -np.sum((counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)))
    return entropy  

print('H(x) = ',round(entropy(target),2))

H(x) =  0.94


In [57]:
# 정보이득 
def InfoGain(data,split_name,target_name):
    total_entropy = entropy(data[target_name]) #before split, original data entropy = MO
    print('Entropy(D) = ', round(total_entropy, 2))
    
    # Calculate entropy_root - average(np.sum(weight*child entropy))
    vals,counts = np.unique(data[split_name],return_counts=True)

    # specific column values matchs outcome. -> if data[split_name]!=vals[i] -> save NaN Thus use dropna() method
    Weighted_Entropy = np.sum([counts[i]/np.sum(counts)*entropy(data.where(data[split_name]==vals[i]).dropna()[target_name])for i in range(len(vals))])
    print('H(', split_name, ') = ', round(Weighted_Entropy, 2))
    
    # 정보이득 계산
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain
 
print('InfoGain( District ) = ', round(InfoGain(dataset, "District", "Outcome"), 2), '\n')
print('InfoGain( House Type ) = ', round(InfoGain(dataset, "House Type", "Outcome"), 2), '\n')
print('InfoGain( Income ) = ', round(InfoGain(dataset, "Income", "Outcome"), 2), '\n')
print('InfoGain( Previous Customer ) = ', round(InfoGain(dataset, "Previous Customer", "Outcome"), 2))

Entropy(D) =  0.94
H( District ) =  0.69
InfoGain( District ) =  0.25 

Entropy(D) =  0.94
H( House Type ) =  0.89
InfoGain( House Type ) =  0.05 

Entropy(D) =  0.94
H( Income ) =  0.79
InfoGain( Income ) =  0.15 

Entropy(D) =  0.94
H( Previous Customer ) =  0.89
InfoGain( Previous Customer ) =  0.05


In [ ]:
def decision_tree(dataset, features, target):
    tree = {parent_node:{}}
    


In [23]:
# ID3 알고리즘
def ID3(data,originaldata,features,target_attribute_name,parent_node_class = None):
 
    # 중지기준 정의
 
    # 1. 대상 속성이 단일값을 가지면: 해당 대상 속성 반환
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
 
    # 2. 데이터가 없을 때: 원본 데이터에서 최대값을 가지는 대상 속성 반환
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])\
               [np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]
 
    # 3. 기술 속성이 없을 때: 부모 노드의 대상 속성 반환
    elif len(features) ==0:
        return parent_node_class
 
    # 트리 성장
    else:
        # 부모노드의 대상 속성 정의(예: Good)
        parent_node_class = np.unique(data[target_attribute_name])\
                            [np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
        
        # 데이터를 분할할 속성 선택
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        # 트리 구조 생성
        tree = {best_feature:{}}
        
        # 최대 정보이득을 보인 기술 속성 제외
        features = [i for i in features if i != best_feature]
        
        # 가지 성장
        for value in np.unique(data[best_feature]):
            # 데이터 분할. dropna(): 결측값을 가진 행, 열 제거
            sub_data = data.where(data[best_feature] == value).dropna()
            
            # ID3 알고리즘
            subtree = ID3(sub_data,data,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
            
        return(tree)

In [24]:
tree = ID3(dataset_pd, dataset_pd, ["District","House Type","Income","Previous Customer"], "Outcome")
from pprint import pprint
pprint(tree)


Entropy(D) =  0.94029
H( District ) =  0.69354
Entropy(D) =  0.94029
H( House Type ) =  0.89031
Entropy(D) =  0.94029
H( Income ) =  0.78845
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
Entropy(D) =  0.94029
H( House Type ) =  0.89031
Entropy(D) =  0.94029
H( Income ) =  0.78845
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
Entropy(D) =  0.94029
H( House Type ) =  0.89031
Entropy(D) =  0.94029
H( Income ) =  0.78845
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
Entropy(D) =  0.94029
H( House Type ) =  0.89031
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
Entropy(D) =  0.94029
H( House Type ) =  0.89031
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
Entropy(D) =  0.94029
H( Previous Customer ) =  0.89216
{'District': {'Rural': 'Responded',
              'Suburban': {'Income': {'High': 'Not responded',
                                      'Low': 'Responded'}},
              'Urban': {'Income': {'High': {'House Type': {'Semi-detached': 'R

In [15]:
# 14 x 5 matrix. 
dataset_np = dataset_pd.to_numpy()

print(dataset_np)

[['Suburban' 'Detached' 'High' 'No' 'Not responded']
 ['Suburban' 'Detached' 'High' 'Yes' 'Not responded']
 ['Rural' 'Detached' 'High' 'No' 'Responed']
 ['Urban' 'Semi-detached' 'High' 'No' 'Responed']
 ['Urban' 'Semi-detached' 'Low' 'No' 'Responed']
 ['Urban' 'Semi-detached' 'Low' 'Yes' 'Not responded']
 ['Rural' 'Semi-detached' 'Low' 'Yes' 'Responed']
 ['Suburban' 'Terrace' 'High' 'No' 'Not responded']
 ['Suburban' 'Semi-detached' 'Low' 'No' 'Responed']
 ['Urban' 'Terrace' 'Low' 'No' 'Responed']
 ['Suburban' 'Terrace' 'Low' 'Yes' 'Responed']
 ['Rural' 'Terrace' 'High' 'Yes' 'Responed']
 ['Rural' 'Detached' 'Low' 'No' 'Responed']
 ['Urban' 'Terrace' 'High' 'Yes' 'Not responed']]


In [ ]:
#Gain = Mo(initial dataset) - M12(Split node)
#Gain is bigger, splited node, creteria is more relevant. -> it can be root node.

#Distinct Gain
